In [1]:
import requests 
import json
from datetime import datetime  # datetime parsing
import pytz  # timezone adjusting
import csv  # for making csv files
import os

In [3]:
username = "___"
password = "___"
save_directory = "/Users/brianmoore/Documents/Jupyter Notebook Projects/3.21 - Python Pract/"

In [11]:
#################################################################
# GET ACCESS TOKEN

# Post credentials
r = requests.post("https://api-7.whoop.com/oauth/token", json={
    "grant_type": "password",
    "issueRefresh": False,
    "password": password,
    "username": username
})

# Exit if fail
if r.status_code != 200:
    print("Fail - Credentials rejected.")
    exit()
else:
    print("Success - Credentials accepted")

# Set userid/token variables
userid = r.json()['user']['id']
access_token = r.json()['access_token']

Success - Credentials accepted


In [19]:
#################################################################
# GET DATA

# Download data
url = 'https://api-7.whoop.com/users/{}/cycles'.format(userid)

params = {
    'start': '2000-01-01T00:00:00.000Z',
    'end': '2030-01-01T00:00:00.000Z'
}

headers = {
    'Authorization': 'bearer {}'.format(access_token)
}

r = requests.get(url, params=params, headers=headers)


if r.status_code != 200:
    print("Fail - User ID / auth token rejected.")
    exit()
else:
    print("Success - User ID / auth token accepted")

Success - User ID / auth token accepted


In [78]:
r.json()

[{'days': ['2019-07-02'],
  'during': {'bounds': '[)',
   'lower': '2019-07-02T05:00:00+00:00',
   'upper': '2019-07-03T03:48:37.35+00:00'},
  'id': 9620538,
  'lastUpdatedAt': '2019-07-03T12:30:47.34013+00:00',
  'predictedEnd': '2019-07-03T05:00:00+00:00',
  'recovery': None,
  'sleep': {'id': None,
   'naps': [],
   'needBreakdown': None,
   'qualityDuration': None,
   'score': None,
   'sleeps': [],
   'state': None},
  'strain': {'averageHeartRate': 66,
   'kilojoules': 952.354,
   'maxHeartRate': 121,
   'rawScore': 5.93946875112433e-05,
   'score': 4.04010540517834,
   'state': None,
   'workouts': []}},
 {'days': ['2019-07-03'],
  'during': {'bounds': '[)',
   'lower': '2019-07-03T03:48:37.35+00:00',
   'upper': '2019-07-04T06:06:11.196+00:00'},
  'id': 9636462,
  'lastUpdatedAt': '2019-07-04T15:13:13.102231+00:00',
  'predictedEnd': '2019-07-04T03:48:37.35+00:00',
  'recovery': {'blackoutUntil': None,
   'calibrating': True,
   'heartRateVariabilityRmssd': 0.0916746,
   'id': 

In [146]:
#################################################################
# PARSE/TRANSFORM DATA



def time_parse(time_string, offset_string):
    # Switch sign on offset
    offset_string = offset_string.replace(
        '-', '+') if offset_string.count('-') else offset_string.replace('+', '-')
    # Remove tz from time and add offset, get to 19 characters
    time_string = time_string[:-(len(time_string) - 19)] + offset_string
    # Parse and format
    oldformat = '%Y-%m-%dT%H:%M:%S%z'
    newformat = '%Y-%m-%d %H:%M:%S'
    return datetime.strptime(time_string, oldformat).astimezone(pytz.utc).strftime(newformat)
    

In [233]:
# Convert data to json
data_raw = r.json()



# Make data list
data_summary = []

# Iterate through data
for d in data_raw:

    # Make record object with default values
    record = {
        'timestamp_measurement': None,
        'rMSSD': None,
        'Recovery Score' : None,
        'Resting Heart Rate' : None,
        'Calories' : None
    }

    if d['recovery'] and 'heartRateVariabilityRmssd' in d['recovery'] and d['recovery'] and 'score' in d['recovery']:
        
        record['timestamp_measurement'] = time_parse(
            d['recovery']['timestamp'],
            d['sleep']['sleeps'][0]['timezoneOffset'])
        record['rMSSD'] = d['recovery']['heartRateVariabilityRmssd'] * 1000.0
        
        
        
        record['Recovery Score'] = d['recovery']['score']
        record['rMSSD'] = d['recovery']['heartRateVariabilityRmssd'] * 1000.0
        record['Resting Heart Rate'] = d['recovery']['restingHeartRate']
        record['Calories'] = d['strain']['kilojoules'] / 4.184

    
    
    


        data_summary.append(record)

In [218]:
sleep_summary = []

for e in data_raw:
    
    sleep_record = {
        'Begin Date' : None,
        'Time in bed' : None,
        'Sleep Pref. Score' : None,
        'Sleep Effic.' : None,
        'Sleep Pref. Score' : None,
        'Disturbances' : None,
        'Light Sleep time' : None,
        'Deep Sleep time' : None,
        'REM Sleep time' : None,
        
    }
    
    for f in e['sleep']['sleeps']:
        sleep_record['Begin Date'] = f['during']['lower']
        sleep_record['Time in bed'] = f['inBedDuration'] / 3.6e+6
        sleep_record['Sleep Pref. Score'] = f['score']
        sleep_record['Sleep Effic.'] = f['sleepEfficiency'] *100
        sleep_record['Disturbances'] = f['disturbanceCount']
        sleep_record['Light Sleep time'] = f['lightSleepDuration'] / 3.6e+6
        sleep_record['Deep Sleep time'] = f['slowWaveSleepDuration'] / 3.6e+6
        sleep_record['REM Sleep time'] = f['remSleepDuration'] / 3.6e+6
        
        
        sleep_summary.append(sleep_record)

In [219]:
for q in sleep_summary:
    print(q)

{'Begin Date': '2019-07-03T03:48:37.35+00:00', 'Time in bed': 8.701928333333333, 'Sleep Pref. Score': 92, 'Sleep Effic.': 89.1589, 'Disturbances': 10, 'Light Sleep time': 3.168528888888889, 'Deep Sleep time': 2.0184966666666666, 'REM Sleep time': 2.500331111111111}
{'Begin Date': '2019-07-04T06:06:11.313+00:00', 'Time in bed': 8.441842222222222, 'Sleep Pref. Score': 88, 'Sleep Effic.': 89.9605, 'Disturbances': 11, 'Light Sleep time': 3.3952644444444444, 'Deep Sleep time': 2.0115469444444445, 'REM Sleep time': 1.9206283333333334}
{'Begin Date': '2019-07-05T04:46:31.589+00:00', 'Time in bed': 8.60366111111111, 'Sleep Pref. Score': 95, 'Sleep Effic.': 96.1717, 'Disturbances': 8, 'Light Sleep time': 3.377088611111111, 'Deep Sleep time': 1.7684908333333333, 'REM Sleep time': 2.966925}
{'Begin Date': '2019-07-06T04:03:01.42+00:00', 'Time in bed': 10.559911944444444, 'Sleep Pref. Score': 100, 'Sleep Effic.': 92.2733, 'Disturbances': 18, 'Light Sleep time': 5.728478055555556, 'Deep Sleep time'

In [234]:
with open(os.path.expanduser(save_directory + 'whooptest.csv'), 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=data_summary[0].keys())
    # Write header
    writer.writeheader()
    # Write rows
    for row in data_summary:
        writer.writerow(row)

print("Success - CSV summary data saved.")

Success - CSV summary data saved.


In [220]:
with open(os.path.expanduser(save_directory + 'whoopsleeptest.csv'), 'w', newline='') as h:
    writer = csv.DictWriter(h, fieldnames=sleep_summary[0].keys())
    # Write header
    writer.writeheader()
    # Write rows
    for row in sleep_summary:
        writer.writerow(row)

print("Success - CSV summary data saved.")

Success - CSV summary data saved.
